<h1 align=center><font size = 5>Predicting Concrete Strength using Regression with Keras</font></h1>

## Introduction

In this notebook,I have used Keras library to build a regression model that predicts the concrete strength given its ingredients values like Cement, BlastFurnace, Flyash,Water Superplasticizer, Coarse aggregate, Fine aggregate and age. I have used regression model because strength is aa continous data in the datset.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>

<a id="item31"></a>

## Download and Clean Dataset

importing the required <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('C:/Users/saisu/Desktop/projects/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Number of data points in the dataset.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. 

Check the dataset for missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength and predictors variable will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>

<a id='item32'></a>

## Import Keras

#### Import the Keras library

In [11]:
import keras

Using TensorFlow backend.


TensorFlow backend was used to install the Keras library.

Import the rest of the packages from the Keras library that need to build regressoin model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    #model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layers with 10 hidden units.

In the below code lets divide the given dataset into train and test data splits with 70-30%. Did both for Train and Test data for further analysis.

In [14]:
import sklearn
from sklearn.model_selection import train_test_split
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3, random_state=42)
predictors_norm_train, predictors_norm_test, target_train, target_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

Checking data before normalising

<a id="item4"></a>

In [15]:
predictors_train.head(5)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
196,194.7,0.0,100.5,165.6,7.5,1006.4,905.9,28
631,325.0,0.0,0.0,184.0,0.0,1063.0,783.0,7
81,318.8,212.5,0.0,155.7,14.3,852.1,880.4,3
526,359.0,19.0,141.0,154.0,10.9,942.0,801.0,3
830,162.0,190.0,148.0,179.0,19.0,838.0,741.0,28


In [16]:
predictors_test.head(5)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
31,266.0,114.0,0.0,228.0,0.0,932.0,670.0,365
109,362.6,189.0,0.0,164.9,11.6,944.7,755.8,7
136,389.9,189.0,0.0,145.9,22.0,944.7,755.8,28
88,362.6,189.0,0.0,164.9,11.6,944.7,755.8,3
918,145.0,0.0,179.0,202.0,8.0,824.0,869.0,28


In [17]:
target_train.head(5)

196    25.72
631    17.54
81     25.20
526    23.64
830    33.76
Name: Strength, dtype: float64

In [18]:
target_test.head(5)

31     52.91
109    55.90
136    74.50
88     35.30
918    10.54
Name: Strength, dtype: float64

Checking data after normalising

In [19]:
predictors_norm_train.head(5)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
196,-0.827393,-0.856472,0.723653,-0.747734,0.216835,0.430603,1.650364,-0.279597
631,0.419421,-0.856472,-0.846733,0.113922,-1.038638,1.158540,0.117485,-0.612034
81,0.360094,1.606458,-0.846733,-1.211343,1.355131,-1.553862,1.332313,-0.675355
526,0.744760,-0.636257,1.356496,-1.290952,0.785983,-0.397651,0.341992,-0.675355
830,-1.140293,1.345678,1.465876,-0.120224,2.141895,-1.735203,-0.406362,-0.279597


In [20]:
predictors_norm_test.head(5)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
31,-0.145138,0.464818,-0.846733,2.174405,-1.038638,-0.526262,-1.291914,5.055221
109,0.779207,1.334087,-0.846733,-0.780515,0.903161,-0.362926,-0.221768,-0.612034
136,1.040436,1.334087,-0.846733,-1.670269,2.644084,-0.362926,-0.221768,-0.279597
88,0.779207,1.334087,-0.846733,-0.780515,0.903161,-0.362926,-0.221768,-0.675355
918,-1.302962,-0.856472,1.950273,0.956847,0.300534,-1.915259,1.190126,-0.279597


In [21]:
target_train.head(5)

196    25.72
631    17.54
81     25.20
526    23.64
830    33.76
Name: Strength, dtype: float64

In [22]:
target_test.head(5)

31     52.91
109    55.90
136    74.50
88     35.30
918    10.54
Name: Strength, dtype: float64

<a id='item34'></a>

## Train and Test the Network

Call the regression model function now to create our model.

In [23]:
# build the model
model = regression_model()

Next, train and test the model at the same time using the *fit* method. leaving out 30% of the data for validation and we will train the model for 50 epochs.

In [24]:
# fit the model
model.fit(predictors_train, target_train, validation_split=0.3, epochs=50, verbose=2)

Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 478077.7440 - val_loss: 389156.7500
Epoch 2/50
 - 0s - loss: 336876.1567 - val_loss: 269405.6969
Epoch 3/50
 - 0s - loss: 232034.9189 - val_loss: 183662.9603
Epoch 4/50
 - 0s - loss: 156896.2401 - val_loss: 122928.5182
Epoch 5/50
 - 0s - loss: 103385.2445 - val_loss: 79459.9541
Epoch 6/50
 - 0s - loss: 65805.0359 - val_loss: 49266.8642
Epoch 7/50
 - 0s - loss: 40050.4744 - val_loss: 29069.3950
Epoch 8/50
 - 0s - loss: 23313.9205 - val_loss: 16226.4950
Epoch 9/50
 - 0s - loss: 12957.1750 - val_loss: 8701.7814
Epoch 10/50
 - 0s - loss: 7064.2473 - val_loss: 4591.9855
Epoch 11/50
 - 0s - loss: 3929.9806 - val_loss: 2532.4714
Epoch 12/50
 - 0s - loss: 2391.7948 - val_loss: 1586.1222
Epoch 13/50
 - 0s - loss: 1715.1407 - val_loss: 1179.9457
Epoch 14/50
 - 0s - loss: 1431.8585 - val_loss: 1018.2253
Epoch 15/50
 - 0s - loss: 1303.9404 - val_loss: 962.2666
Epoch 16/50
 - 0s - loss: 1253.5251 - val_loss: 935.7182
Epoch 17/50

Predict the strengths of test data in the dataset using the built model and data before normalising

In [25]:
target_predicted=model.predict(predictors_test)
target_predicted

array([[ 7.932184 ],
       [30.204905 ],
       [27.962595 ],
       [30.496454 ],
       [58.153038 ],
       [16.98282  ],
       [25.110155 ],
       [36.862045 ],
       [41.910652 ],
       [32.92141  ],
       [47.66926  ],
       [51.800278 ],
       [16.491755 ],
       [25.316446 ],
       [36.69651  ],
       [29.033396 ],
       [14.754244 ],
       [39.372307 ],
       [20.496996 ],
       [35.49433  ],
       [42.098335 ],
       [36.952263 ],
       [30.311892 ],
       [37.665985 ],
       [42.233215 ],
       [38.50051  ],
       [67.92575  ],
       [41.58178  ],
       [36.653206 ],
       [44.4461   ],
       [63.45205  ],
       [37.440315 ],
       [26.05099  ],
       [47.258774 ],
       [36.53631  ],
       [29.103342 ],
       [42.809055 ],
       [23.340876 ],
       [34.440712 ],
       [38.192963 ],
       [60.409554 ],
       [47.125237 ],
       [36.39379  ],
       [26.169779 ],
       [43.680695 ],
       [23.358255 ],
       [19.553339 ],
       [19.35

In [26]:
results = model.evaluate(predictors_test,target_test,verbose=0)
print('test loss, test acc:', results)
#model.compile(optimizer='adam', loss='mean_squared_error')

test loss, test acc: 654.7923816075988


In [27]:
from sklearn.metrics import mean_squared_error
mean_squared_error(target_test, target_predicted)

654.7923897661105

Loop over the model 50 times with changing the random state to get different samples, each time.

In [28]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
mean_squared_error_array=[]
for i in range(50):
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    #predictors_norm_train, predictors_norm_test, target_train, target_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model = regression_model()
    model.fit(predictors_train, target_train, validation_split=0.3, epochs=50, verbose=2)
    target_predicted=model.predict(predictors_test)
    target_predicted
    results = model.evaluate(predictors_test,target_test,verbose=0)
    #print('test loss, test acc:', results)
    mean_squared_error_array.append(mean_squared_error(target_test, target_predicted))
    print(i)

Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 4190.9173 - val_loss: 2982.7478
Epoch 2/50
 - 0s - loss: 3192.0353 - val_loss: 2502.0598
Epoch 3/50
 - 0s - loss: 2605.3359 - val_loss: 2005.6926
Epoch 4/50
 - 0s - loss: 2108.7565 - val_loss: 1652.3620
Epoch 5/50
 - 0s - loss: 1721.3188 - val_loss: 1380.0129
Epoch 6/50
 - 0s - loss: 1398.2050 - val_loss: 1111.1289
Epoch 7/50
 - 0s - loss: 1141.7531 - val_loss: 923.2848
Epoch 8/50
 - 0s - loss: 973.0962 - val_loss: 853.5743
Epoch 9/50
 - 0s - loss: 817.3837 - val_loss: 697.3676
Epoch 10/50
 - 0s - loss: 717.0258 - val_loss: 617.8591
Epoch 11/50
 - 0s - loss: 619.4503 - val_loss: 532.9458
Epoch 12/50
 - 0s - loss: 555.7691 - val_loss: 480.5653
Epoch 13/50
 - 0s - loss: 493.3308 - val_loss: 433.8976
Epoch 14/50
 - 0s - loss: 430.8108 - val_loss: 405.9486
Epoch 15/50
 - 0s - loss: 389.8576 - val_loss: 367.6017
Epoch 16/50
 - 0s - loss: 353.1557 - val_loss: 335.8707
Epoch 17/50
 - 0s - loss: 325.8699 - val_loss: 316.728

Epoch 43/50
 - 0s - loss: 187.5083 - val_loss: 196.2488
Epoch 44/50
 - 0s - loss: 184.5040 - val_loss: 191.9234
Epoch 45/50
 - 0s - loss: 181.3060 - val_loss: 188.5345
Epoch 46/50
 - 0s - loss: 178.4994 - val_loss: 184.6195
Epoch 47/50
 - 0s - loss: 175.5925 - val_loss: 181.0730
Epoch 48/50
 - 0s - loss: 172.8634 - val_loss: 177.5395
Epoch 49/50
 - 0s - loss: 170.2549 - val_loss: 174.3541
Epoch 50/50
 - 0s - loss: 167.7951 - val_loss: 171.3638
2
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 233580.1059 - val_loss: 176422.7739
Epoch 2/50
 - 0s - loss: 150109.3555 - val_loss: 106803.8656
Epoch 3/50
 - 0s - loss: 87936.8015 - val_loss: 57135.6617
Epoch 4/50
 - 0s - loss: 43387.7185 - val_loss: 24461.5893
Epoch 5/50
 - 0s - loss: 18128.5417 - val_loss: 10618.2301
Epoch 6/50
 - 0s - loss: 9193.7607 - val_loss: 7757.0894
Epoch 7/50
 - 0s - loss: 7731.0008 - val_loss: 7266.2207
Epoch 8/50
 - 0s - loss: 7202.9984 - val_loss: 6760.3688
Epoch 9/50
 - 0s - loss: 6695.5450

Epoch 36/50
 - 0s - loss: 224.7629 - val_loss: 203.9001
Epoch 37/50
 - 0s - loss: 219.8059 - val_loss: 202.3098
Epoch 38/50
 - 0s - loss: 216.1188 - val_loss: 196.9439
Epoch 39/50
 - 0s - loss: 210.5353 - val_loss: 187.9923
Epoch 40/50
 - 0s - loss: 206.1636 - val_loss: 189.3971
Epoch 41/50
 - 0s - loss: 201.8972 - val_loss: 185.0874
Epoch 42/50
 - 0s - loss: 198.4686 - val_loss: 178.4915
Epoch 43/50
 - 0s - loss: 194.0618 - val_loss: 178.2712
Epoch 44/50
 - 0s - loss: 191.5911 - val_loss: 177.4863
Epoch 45/50
 - 0s - loss: 187.5123 - val_loss: 166.8692
Epoch 46/50
 - 0s - loss: 185.7170 - val_loss: 166.7422
Epoch 47/50
 - 0s - loss: 183.0877 - val_loss: 171.9491
Epoch 48/50
 - 0s - loss: 179.9283 - val_loss: 159.5672
Epoch 49/50
 - 0s - loss: 177.9245 - val_loss: 160.4398
Epoch 50/50
 - 0s - loss: 174.3436 - val_loss: 153.6920
5
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 241685.6615 - val_loss: 170387.6577
Epoch 2/50
 - 0s - loss: 129580.3986 - val_loss: 83

Epoch 27/50
 - 0s - loss: 644.1508 - val_loss: 637.3483
Epoch 28/50
 - 0s - loss: 562.6985 - val_loss: 557.3842
Epoch 29/50
 - 0s - loss: 490.8914 - val_loss: 487.2525
Epoch 30/50
 - 0s - loss: 428.0270 - val_loss: 423.6977
Epoch 31/50
 - 0s - loss: 379.3250 - val_loss: 376.8556
Epoch 32/50
 - 0s - loss: 333.1740 - val_loss: 324.1710
Epoch 33/50
 - 0s - loss: 287.7917 - val_loss: 287.0395
Epoch 34/50
 - 0s - loss: 254.6178 - val_loss: 251.4281
Epoch 35/50
 - 0s - loss: 224.7322 - val_loss: 222.7328
Epoch 36/50
 - 0s - loss: 199.4198 - val_loss: 200.4330
Epoch 37/50
 - 0s - loss: 179.0505 - val_loss: 178.9305
Epoch 38/50
 - 0s - loss: 161.7221 - val_loss: 164.8778
Epoch 39/50
 - 0s - loss: 147.9112 - val_loss: 148.8248
Epoch 40/50
 - 0s - loss: 136.3674 - val_loss: 138.4441
Epoch 41/50
 - 0s - loss: 126.6197 - val_loss: 131.2173
Epoch 42/50
 - 0s - loss: 118.2930 - val_loss: 121.3685
Epoch 43/50
 - 0s - loss: 112.4441 - val_loss: 115.8876
Epoch 44/50
 - 0s - loss: 107.2541 - val_loss: 1

Epoch 18/50
 - 0s - loss: 2031.6253 - val_loss: 2117.7339
Epoch 19/50
 - 0s - loss: 1938.7033 - val_loss: 2028.8212
Epoch 20/50
 - 0s - loss: 1856.4568 - val_loss: 1943.9059
Epoch 21/50
 - 0s - loss: 1775.3028 - val_loss: 1870.1220
Epoch 22/50
 - 0s - loss: 1703.6307 - val_loss: 1795.3907
Epoch 23/50
 - 0s - loss: 1620.6755 - val_loss: 1703.5630
Epoch 24/50
 - 0s - loss: 1553.9825 - val_loss: 1633.9171
Epoch 25/50
 - 0s - loss: 1479.7724 - val_loss: 1551.1692
Epoch 26/50
 - 0s - loss: 1413.0548 - val_loss: 1488.3850
Epoch 27/50
 - 0s - loss: 1350.9684 - val_loss: 1439.1174
Epoch 28/50
 - 0s - loss: 1286.9146 - val_loss: 1375.4527
Epoch 29/50
 - 0s - loss: 1234.9515 - val_loss: 1293.2265
Epoch 30/50
 - 0s - loss: 1172.7223 - val_loss: 1250.8552
Epoch 31/50
 - 0s - loss: 1117.8835 - val_loss: 1192.0955
Epoch 32/50
 - 0s - loss: 1067.7345 - val_loss: 1130.8734
Epoch 33/50
 - 0s - loss: 1015.1518 - val_loss: 1088.0007
Epoch 34/50
 - 0s - loss: 968.7114 - val_loss: 1041.9880
Epoch 35/50
 - 

...

Epoch 45/50
 - 0s - loss: 549.6016 - val_loss: 643.0378
Epoch 46/50
 - 0s - loss: 529.7851 - val_loss: 624.7029
Epoch 47/50
 - 0s - loss: 508.4318 - val_loss: 598.9181
Epoch 48/50
 - 0s - loss: 489.5868 - val_loss: 578.3585
Epoch 49/50
 - 0s - loss: 473.8443 - val_loss: 560.6122
Epoch 50/50
 - 0s - loss: 457.5135 - val_loss: 538.1651
36
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 118586.6229 - val_loss: 65142.6741
Epoch 2/50
 - 0s - loss: 39588.2944 - val_loss: 16166.5396
Epoch 3/50
 - 0s - loss: 8596.4630 - val_loss: 2836.7609
Epoch 4/50
 - 0s - loss: 2092.2678 - val_loss: 1475.7001
Epoch 5/50
 - 0s - loss: 1641.0339 - val_loss: 1527.4008
Epoch 6/50
 - 0s - loss: 1589.3574 - val_loss: 1406.8205
Epoch 7/50
 - 0s - loss: 1482.9130 - val_loss: 1323.1935
Epoch 8/50
 - 0s - loss: 1410.5109 - val_loss: 1257.7816
Epoch 9/50
 - 0s - loss: 1346.3399 - val_loss: 1204.1225
Epoch 10/50
 - 0s - loss: 1278.5816 - val_loss: 1148.6365
Epoch 11/50
 - 0s - loss: 1213.6120 - v

Epoch 37/50
 - 0s - loss: 573.3367 - val_loss: 502.3536
Epoch 38/50
 - 0s - loss: 543.9839 - val_loss: 482.0860
Epoch 39/50
 - 0s - loss: 518.9016 - val_loss: 463.3480
Epoch 40/50
 - 0s - loss: 496.4399 - val_loss: 446.5250
Epoch 41/50
 - 0s - loss: 476.0733 - val_loss: 432.2874
Epoch 42/50
 - 0s - loss: 458.8000 - val_loss: 419.4270
Epoch 43/50
 - 0s - loss: 442.7286 - val_loss: 407.7844
Epoch 44/50
 - 0s - loss: 429.4678 - val_loss: 397.7999
Epoch 45/50
 - 0s - loss: 415.9344 - val_loss: 387.9370
Epoch 46/50
 - 0s - loss: 404.4054 - val_loss: 379.4446
Epoch 47/50
 - 0s - loss: 393.7588 - val_loss: 371.0441
Epoch 48/50
 - 0s - loss: 383.5421 - val_loss: 363.6714
Epoch 49/50
 - 0s - loss: 373.2811 - val_loss: 356.2802
Epoch 50/50
 - 0s - loss: 364.1132 - val_loss: 350.1584
39
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 550852.5898 - val_loss: 462826.9999
Epoch 2/50
 - 0s - loss: 392515.0203 - val_loss: 327975.2334
Epoch 3/50
 - 0s - loss: 278343.4774 - val_lo

Epoch 26/50
 - 0s - loss: 2870.9145 - val_loss: 2824.5369
Epoch 27/50
 - 0s - loss: 2783.1163 - val_loss: 2752.4157
Epoch 28/50
 - 0s - loss: 2693.2791 - val_loss: 2657.0658
Epoch 29/50
 - 0s - loss: 2610.2419 - val_loss: 2586.8210
Epoch 30/50
 - 0s - loss: 2525.6281 - val_loss: 2509.2731
Epoch 31/50
 - 0s - loss: 2445.0902 - val_loss: 2444.6300
Epoch 32/50
 - 0s - loss: 2370.0580 - val_loss: 2366.9820
Epoch 33/50
 - 0s - loss: 2293.1577 - val_loss: 2299.3516
Epoch 34/50
 - 0s - loss: 2217.3564 - val_loss: 2236.8598
Epoch 35/50
 - 0s - loss: 2147.9342 - val_loss: 2169.0588
Epoch 36/50
 - 0s - loss: 2079.1046 - val_loss: 2119.3848
Epoch 37/50
 - 0s - loss: 2013.3739 - val_loss: 2048.3412
Epoch 38/50
 - 0s - loss: 1949.3818 - val_loss: 1986.9876
Epoch 39/50
 - 0s - loss: 1885.5914 - val_loss: 1933.7550
Epoch 40/50
 - 0s - loss: 1827.3474 - val_loss: 1886.4759
Epoch 41/50
 - 0s - loss: 1769.1416 - val_loss: 1827.2621
Epoch 42/50
 - 0s - loss: 1712.9375 - val_loss: 1779.3025
Epoch 43/50
 -

Epoch 17/50
 - 0s - loss: 1275.5333 - val_loss: 1437.8888
Epoch 18/50
 - 0s - loss: 1175.3123 - val_loss: 1315.4116
Epoch 19/50
 - 0s - loss: 1084.3656 - val_loss: 1209.0271
Epoch 20/50
 - 0s - loss: 995.3936 - val_loss: 1111.0151
Epoch 21/50
 - 0s - loss: 920.5179 - val_loss: 1021.3617
Epoch 22/50
 - 0s - loss: 850.7206 - val_loss: 944.4585
Epoch 23/50
 - 0s - loss: 789.3660 - val_loss: 870.9018
Epoch 24/50
 - 0s - loss: 732.0948 - val_loss: 807.0146
Epoch 25/50
 - 0s - loss: 680.8603 - val_loss: 748.8436
Epoch 26/50
 - 0s - loss: 632.6476 - val_loss: 696.0670
Epoch 27/50
 - 0s - loss: 588.1718 - val_loss: 647.3522
Epoch 28/50
 - 0s - loss: 548.8607 - val_loss: 606.7361
Epoch 29/50
 - 0s - loss: 513.5328 - val_loss: 569.7789
Epoch 30/50
 - 0s - loss: 482.6661 - val_loss: 531.5878
Epoch 31/50
 - 0s - loss: 450.6529 - val_loss: 502.8297
Epoch 32/50
 - 0s - loss: 427.0688 - val_loss: 472.9259
Epoch 33/50
 - 0s - loss: 403.4019 - val_loss: 446.9246
Epoch 34/50
 - 0s - loss: 382.8353 - val

Epoch 10/50
 - 0s - loss: 2555.2124 - val_loss: 2682.8722
Epoch 11/50
 - 0s - loss: 2415.0288 - val_loss: 2550.5454
Epoch 12/50
 - 0s - loss: 2277.2786 - val_loss: 2411.8023
Epoch 13/50
 - 0s - loss: 2136.9171 - val_loss: 2283.6151
Epoch 14/50
 - 0s - loss: 2002.8246 - val_loss: 2159.3178
Epoch 15/50
 - 0s - loss: 1882.7921 - val_loss: 2044.4670
Epoch 16/50
 - 0s - loss: 1771.4964 - val_loss: 1935.6084
Epoch 17/50
 - 0s - loss: 1674.2662 - val_loss: 1839.4688
Epoch 18/50
 - 0s - loss: 1584.2055 - val_loss: 1744.2516
Epoch 19/50
 - 0s - loss: 1500.8820 - val_loss: 1661.2885
Epoch 20/50
 - 0s - loss: 1421.9268 - val_loss: 1584.7083
Epoch 21/50
 - 0s - loss: 1348.2761 - val_loss: 1515.3528
Epoch 22/50
 - 0s - loss: 1283.4774 - val_loss: 1439.4500
Epoch 23/50
 - 0s - loss: 1224.4901 - val_loss: 1387.4440
Epoch 24/50
 - 0s - loss: 1164.1141 - val_loss: 1315.3852
Epoch 25/50
 - 0s - loss: 1112.7713 - val_loss: 1261.7813
Epoch 26/50
 - 0s - loss: 1065.9874 - val_loss: 1213.7097
Epoch 27/50
 -

In [29]:
mean_squared_error_array

[108.86779568653533,
 1204.7016220715216,
 192.84448095364252,
 373.6277253909298,
 150.0228722109658,
 165.48395367090353,
 1373.279895931734,
 574.9268919198054,
 128.44832365110952,
 467.59323658736656,
 899.9655715620999,
 456.94065922338564,
 228.75035478793478,
 384.85132890753846,
 549.9962040352603,
 2091.518349335264,
 341.3157417239012,
 1655.8105887950642,
 218.05476253824284,
 145.64742435869013,
 1131.45062393386,
 424.93663994939766,
 336.76756442374307,
 133.40262532125251,
 471.3396170033565,
 142.56970745961922,
 249.03688556654265,
 828.3826881724214,
 245.10445406852924,
 658.762957384414,
 494.938662099164,
 2363.7352387565215,
 119.81703606053436,
 520.6757075467281,
 134.7811913093664,
 552.8991857033933,
 490.4399382245791,
 332.48306404145717,
 126.69705803162566,
 264.25057130895806,
 2064.554677330616,
 621.7172834214232,
 1505.357888513084,
 460.3740434288018,
 335.66249909712445,
 267.16725783169016,
 173.92698527414134,
 220.17884187238155,
 465.21289327398

In [30]:
from statistics import mean,pstdev
print("mean:",mean(mean_squared_error_array))

mean: 561.1341654461006


In [31]:
print("sd:",pstdev(mean_squared_error_array))

sd: 544.5242683332228


Building the same model on normalised data

In [32]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
mean_squared_error_array2=[]
for i in range(50):
    #predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    predictors_norm_train, predictors_norm_test, target_train, target_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model = regression_model()
    model.fit(predictors_norm_train, target_train, validation_split=0.3, epochs=50, verbose=2)
    target_predicted=model.predict(predictors_test)
    target_predicted
    results = model.evaluate(predictors_norm_test,target_test,verbose=0)
    #print('test loss, test acc:', results)
    mean_squared_error_array2.append(mean_squared_error(target_test, target_predicted))
    print(i)
mean_squared_error_array2
mean(mean_squared_error_array2)

Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 1560.7705 - val_loss: 1647.1247
Epoch 2/50
 - 0s - loss: 1551.0961 - val_loss: 1637.1384
Epoch 3/50
 - 0s - loss: 1541.4570 - val_loss: 1627.0442
Epoch 4/50
 - 0s - loss: 1531.7487 - val_loss: 1616.8170
Epoch 5/50
 - 0s - loss: 1521.8881 - val_loss: 1606.3459
Epoch 6/50
 - 0s - loss: 1511.7515 - val_loss: 1595.6228
Epoch 7/50
 - 0s - loss: 1501.2961 - val_loss: 1584.4949
Epoch 8/50
 - 0s - loss: 1490.4937 - val_loss: 1572.9378
Epoch 9/50
 - 0s - loss: 1479.0759 - val_loss: 1561.1690
Epoch 10/50
 - 0s - loss: 1467.4387 - val_loss: 1548.6832
Epoch 11/50
 - 0s - loss: 1455.2742 - val_loss: 1535.4944
Epoch 12/50
 - 0s - loss: 1442.1727 - val_loss: 1521.9546
Epoch 13/50
 - 0s - loss: 1428.7696 - val_loss: 1507.5598
Epoch 14/50
 - 0s - loss: 1414.5468 - val_loss: 1492.3999
Epoch 15/50
 - 0s - loss: 1399.6995 - val_loss: 1476.4941
Epoch 16/50
 - 0s - loss: 1384.0313 - val_loss: 1459.8403
Epoch 17/50
 - 0s - loss: 1367.6008

Epoch 42/50
 - 0s - loss: 930.0719 - val_loss: 842.9025
Epoch 43/50
 - 0s - loss: 903.9597 - val_loss: 818.8364
Epoch 44/50
 - 0s - loss: 878.7003 - val_loss: 794.3455
Epoch 45/50
 - 0s - loss: 852.4507 - val_loss: 770.8197
Epoch 46/50
 - 0s - loss: 827.1428 - val_loss: 747.3739
Epoch 47/50
 - 0s - loss: 802.0810 - val_loss: 724.0903
Epoch 48/50
 - 0s - loss: 777.3561 - val_loss: 700.8872
Epoch 49/50
 - 0s - loss: 752.4388 - val_loss: 678.7991
Epoch 50/50
 - 0s - loss: 728.4101 - val_loss: 657.2896
2
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 1509.8467 - val_loss: 1671.0341
Epoch 2/50
 - 0s - loss: 1497.2042 - val_loss: 1656.7242
Epoch 3/50
 - 0s - loss: 1484.4971 - val_loss: 1642.8312
Epoch 4/50
 - 0s - loss: 1471.8232 - val_loss: 1629.2457
Epoch 5/50
 - 0s - loss: 1459.3962 - val_loss: 1614.9414
Epoch 6/50
 - 0s - loss: 1446.5490 - val_loss: 1600.9171
Epoch 7/50
 - 0s - loss: 1433.6356 - val_loss: 1586.5415
Epoch 8/50
 - 0s - loss: 1420.4888 - val_loss: 15

Epoch 33/50
 - 0s - loss: 1130.3960 - val_loss: 1036.4380
Epoch 34/50
 - 0s - loss: 1109.0241 - val_loss: 1018.7030
Epoch 35/50
 - 0s - loss: 1088.0790 - val_loss: 1000.4810
Epoch 36/50
 - 0s - loss: 1066.3010 - val_loss: 982.7154
Epoch 37/50
 - 0s - loss: 1044.7212 - val_loss: 964.7859
Epoch 38/50
 - 0s - loss: 1023.6031 - val_loss: 946.6012
Epoch 39/50
 - 0s - loss: 1002.0052 - val_loss: 928.6185
Epoch 40/50
 - 0s - loss: 981.0003 - val_loss: 910.4782
Epoch 41/50
 - 0s - loss: 959.5092 - val_loss: 893.0037
Epoch 42/50
 - 0s - loss: 938.5447 - val_loss: 875.0525
Epoch 43/50
 - 0s - loss: 917.5795 - val_loss: 857.4156
Epoch 44/50
 - 0s - loss: 896.7112 - val_loss: 839.8949
Epoch 45/50
 - 0s - loss: 875.9329 - val_loss: 822.6468
Epoch 46/50
 - 0s - loss: 855.7561 - val_loss: 805.1256
Epoch 47/50
 - 0s - loss: 835.7916 - val_loss: 787.7530
Epoch 48/50
 - 0s - loss: 815.5229 - val_loss: 770.8762
Epoch 49/50
 - 0s - loss: 796.2542 - val_loss: 753.6959
Epoch 50/50
 - 0s - loss: 776.6315 - v

Epoch 24/50
 - 0s - loss: 1163.1471 - val_loss: 1260.3359
Epoch 25/50
 - 0s - loss: 1144.9013 - val_loss: 1242.2689
Epoch 26/50
 - 0s - loss: 1126.7112 - val_loss: 1224.3551
Epoch 27/50
 - 0s - loss: 1108.8088 - val_loss: 1205.7519
Epoch 28/50
 - 0s - loss: 1090.4732 - val_loss: 1187.3524
Epoch 29/50
 - 0s - loss: 1072.4500 - val_loss: 1168.1810
Epoch 30/50
 - 0s - loss: 1053.9207 - val_loss: 1149.3937
Epoch 31/50
 - 0s - loss: 1035.6047 - val_loss: 1130.7712
Epoch 32/50
 - 0s - loss: 1017.3250 - val_loss: 1111.8045
Epoch 33/50
 - 0s - loss: 999.1500 - val_loss: 1092.6497
Epoch 34/50
 - 0s - loss: 980.7330 - val_loss: 1074.0327
Epoch 35/50
 - 0s - loss: 962.6474 - val_loss: 1055.2904
Epoch 36/50
 - 0s - loss: 944.8447 - val_loss: 1036.3891
Epoch 37/50
 - 0s - loss: 926.8206 - val_loss: 1018.3077
Epoch 38/50
 - 0s - loss: 909.5639 - val_loss: 999.5071
Epoch 39/50
 - 0s - loss: 891.7242 - val_loss: 981.6042
Epoch 40/50
 - 0s - loss: 874.6010 - val_loss: 963.8086
Epoch 41/50
 - 0s - loss:

Epoch 15/50
 - 0s - loss: 1353.0252 - val_loss: 1572.4478
Epoch 16/50
 - 0s - loss: 1336.4620 - val_loss: 1554.0125
Epoch 17/50
 - 0s - loss: 1318.8300 - val_loss: 1535.2894
Epoch 18/50
 - 0s - loss: 1300.7287 - val_loss: 1515.6241
Epoch 19/50
 - 0s - loss: 1282.0409 - val_loss: 1495.1395
Epoch 20/50
 - 0s - loss: 1262.5992 - val_loss: 1474.0287
Epoch 21/50
 - 0s - loss: 1242.4638 - val_loss: 1452.5076
Epoch 22/50
 - 0s - loss: 1222.1430 - val_loss: 1429.9302
Epoch 23/50
 - 0s - loss: 1200.6636 - val_loss: 1407.5236
Epoch 24/50
 - 0s - loss: 1179.5074 - val_loss: 1383.8744
Epoch 25/50
 - 0s - loss: 1157.2260 - val_loss: 1360.5877
Epoch 26/50
 - 0s - loss: 1135.1495 - val_loss: 1336.5602
Epoch 27/50
 - 0s - loss: 1112.7999 - val_loss: 1311.7115
Epoch 28/50
 - 0s - loss: 1089.7275 - val_loss: 1286.9058
Epoch 29/50
 - 0s - loss: 1066.5734 - val_loss: 1261.8327
Epoch 30/50
 - 0s - loss: 1043.0510 - val_loss: 1236.5509
Epoch 31/50
 - 0s - loss: 1019.6681 - val_loss: 1210.5249
Epoch 32/50
 -

...

 - 0s - loss: 1264.5420 - val_loss: 1137.3267
Epoch 27/50
 - 0s - loss: 1245.8044 - val_loss: 1120.5847
Epoch 28/50
 - 0s - loss: 1225.9712 - val_loss: 1103.7975
Epoch 29/50
 - 0s - loss: 1206.2601 - val_loss: 1086.6198
Epoch 30/50
 - 0s - loss: 1185.9514 - val_loss: 1069.1375
Epoch 31/50
 - 0s - loss: 1165.3582 - val_loss: 1051.2975
Epoch 32/50
 - 0s - loss: 1144.1967 - val_loss: 1033.4060
Epoch 33/50
 - 0s - loss: 1123.1252 - val_loss: 1015.2419
Epoch 34/50
 - 0s - loss: 1101.2027 - val_loss: 997.4221
Epoch 35/50
 - 0s - loss: 1079.5832 - val_loss: 979.2765
Epoch 36/50
 - 0s - loss: 1057.9183 - val_loss: 960.5990
Epoch 37/50
 - 0s - loss: 1035.3971 - val_loss: 942.1996
Epoch 38/50
 - 0s - loss: 1013.2956 - val_loss: 923.6490
Epoch 39/50
 - 0s - loss: 991.5713 - val_loss: 904.8142
Epoch 40/50
 - 0s - loss: 969.1146 - val_loss: 886.2941
Epoch 41/50
 - 0s - loss: 947.0368 - val_loss: 867.9942
Epoch 42/50
 - 0s - loss: 925.0883 - val_loss: 849.9918
Epoch 43/50
 - 0s - loss: 903.8121 - va

 - 0s - loss: 1438.8827 - val_loss: 1319.8162
Epoch 18/50
 - 0s - loss: 1423.7070 - val_loss: 1305.1286
Epoch 19/50
 - 0s - loss: 1407.6256 - val_loss: 1290.3922
Epoch 20/50
 - 0s - loss: 1391.2130 - val_loss: 1275.2860
Epoch 21/50
 - 0s - loss: 1374.3856 - val_loss: 1259.4372
Epoch 22/50
 - 0s - loss: 1356.9059 - val_loss: 1243.1991
Epoch 23/50
 - 0s - loss: 1338.7997 - val_loss: 1226.6716
Epoch 24/50
 - 0s - loss: 1320.2498 - val_loss: 1209.5994
Epoch 25/50
 - 0s - loss: 1301.2124 - val_loss: 1191.9844
Epoch 26/50
 - 0s - loss: 1281.6168 - val_loss: 1173.7166
Epoch 27/50
 - 0s - loss: 1261.2737 - val_loss: 1155.2102
Epoch 28/50
 - 0s - loss: 1240.4882 - val_loss: 1136.1960
Epoch 29/50
 - 0s - loss: 1219.2909 - val_loss: 1116.3650
Epoch 30/50
 - 0s - loss: 1197.0860 - val_loss: 1096.4209
Epoch 31/50
 - 0s - loss: 1174.8848 - val_loss: 1075.6186
Epoch 32/50
 - 0s - loss: 1151.5938 - val_loss: 1054.7919
Epoch 33/50
 - 0s - loss: 1128.3753 - val_loss: 1033.3739
Epoch 34/50
 - 0s - loss: 

 - 0s - loss: 1498.1266 - val_loss: 1456.6234
Epoch 9/50
 - 0s - loss: 1484.9224 - val_loss: 1445.3557
Epoch 10/50
 - 0s - loss: 1472.1068 - val_loss: 1433.4859
Epoch 11/50
 - 0s - loss: 1458.4880 - val_loss: 1421.7020
Epoch 12/50
 - 0s - loss: 1444.9057 - val_loss: 1409.5228
Epoch 13/50
 - 0s - loss: 1430.5680 - val_loss: 1397.3614
Epoch 14/50
 - 0s - loss: 1416.2694 - val_loss: 1384.6965
Epoch 15/50
 - 0s - loss: 1401.1290 - val_loss: 1371.9705
Epoch 16/50
 - 0s - loss: 1385.8040 - val_loss: 1358.8264
Epoch 17/50
 - 0s - loss: 1370.0833 - val_loss: 1345.4426
Epoch 18/50
 - 0s - loss: 1353.7627 - val_loss: 1331.6929
Epoch 19/50
 - 0s - loss: 1337.1360 - val_loss: 1317.7562
Epoch 20/50
 - 0s - loss: 1319.9652 - val_loss: 1303.7195
Epoch 21/50
 - 0s - loss: 1302.8743 - val_loss: 1289.2703
Epoch 22/50
 - 0s - loss: 1285.1472 - val_loss: 1274.7838
Epoch 23/50
 - 0s - loss: 1266.9411 - val_loss: 1260.1456
Epoch 24/50
 - 0s - loss: 1248.6825 - val_loss: 1244.9569
Epoch 25/50
 - 0s - loss: 1

Epoch 50/50
 - 0s - loss: 507.6687 - val_loss: 471.7993
47
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 1600.2296 - val_loss: 1375.6230
Epoch 2/50
 - 0s - loss: 1587.7670 - val_loss: 1364.3244
Epoch 3/50
 - 0s - loss: 1575.1876 - val_loss: 1352.9996
Epoch 4/50
 - 0s - loss: 1562.6092 - val_loss: 1341.3048
Epoch 5/50
 - 0s - loss: 1549.6777 - val_loss: 1329.4864
Epoch 6/50
 - 0s - loss: 1536.6771 - val_loss: 1317.3161
Epoch 7/50
 - 0s - loss: 1523.2365 - val_loss: 1305.0695
Epoch 8/50
 - 0s - loss: 1509.7264 - val_loss: 1292.5160
Epoch 9/50
 - 0s - loss: 1495.8768 - val_loss: 1279.5968
Epoch 10/50
 - 0s - loss: 1481.5349 - val_loss: 1266.4942
Epoch 11/50
 - 0s - loss: 1466.9768 - val_loss: 1252.9136
Epoch 12/50
 - 0s - loss: 1451.8366 - val_loss: 1239.1486
Epoch 13/50
 - 0s - loss: 1436.4789 - val_loss: 1224.9599
Epoch 14/50
 - 0s - loss: 1420.6725 - val_loss: 1210.4378
Epoch 15/50
 - 0s - loss: 1404.5797 - val_loss: 1195.4465
Epoch 16/50
 - 0s - loss: 1387.845

7125312.897080838

In [33]:
mean_squared_error_array2
mean(mean_squared_error_array2)

7125312.897080838

In [34]:
print("sd:",pstdev(mean_squared_error_array2))

sd: 5740924.124464269


Model built on normalised data for 100 epochs

In [35]:
mean_squared_error_array3=[]
for i in range(50):
    #predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    predictors_norm_train, predictors_norm_test, target_train, target_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model = regression_model()
    model.fit(predictors_norm_train, target_train, validation_split=0.3, epochs=100, verbose=2)
    target_predicted=model.predict(predictors_test)
    target_predicted
    results = model.evaluate(predictors_norm_test,target_test,verbose=0)
    #print('test loss, test acc:', results)
    mean_squared_error_array3.append(mean_squared_error(target_test, target_predicted))
    print(i)
mean_squared_error_array3

Train on 504 samples, validate on 217 samples
Epoch 1/100
 - 0s - loss: 1495.4922 - val_loss: 1572.7897
Epoch 2/100
 - 0s - loss: 1485.0201 - val_loss: 1561.3124
Epoch 3/100
 - 0s - loss: 1474.2567 - val_loss: 1549.5674
Epoch 4/100
 - 0s - loss: 1463.3649 - val_loss: 1537.6013
Epoch 5/100
 - 0s - loss: 1452.2392 - val_loss: 1525.2027
Epoch 6/100
 - 0s - loss: 1440.5908 - val_loss: 1512.3501
Epoch 7/100
 - 0s - loss: 1428.4654 - val_loss: 1498.7516
Epoch 8/100
 - 0s - loss: 1415.7266 - val_loss: 1484.5235
Epoch 9/100
 - 0s - loss: 1402.3422 - val_loss: 1469.6771
Epoch 10/100
 - 0s - loss: 1388.6023 - val_loss: 1453.4753
Epoch 11/100
 - 0s - loss: 1373.8081 - val_loss: 1436.8047
Epoch 12/100
 - 0s - loss: 1358.3626 - val_loss: 1419.6373
Epoch 13/100
 - 0s - loss: 1342.5012 - val_loss: 1401.3412
Epoch 14/100
 - 0s - loss: 1325.7425 - val_loss: 1382.1970
Epoch 15/100
 - 0s - loss: 1308.1514 - val_loss: 1362.7038
Epoch 16/100
 - 0s - loss: 1290.1389 - val_loss: 1342.1320
Epoch 17/100
 - 0s 

Epoch 42/100
 - 0s - loss: 872.2119 - val_loss: 940.6677
Epoch 43/100
 - 0s - loss: 847.6899 - val_loss: 917.3317
Epoch 44/100
 - 0s - loss: 823.0657 - val_loss: 894.4379
Epoch 45/100
 - 0s - loss: 799.0816 - val_loss: 870.7662
Epoch 46/100
 - 0s - loss: 774.7582 - val_loss: 847.3868
Epoch 47/100
 - 0s - loss: 750.6986 - val_loss: 824.1042
Epoch 48/100
 - 0s - loss: 726.6018 - val_loss: 801.3250
Epoch 49/100
 - 0s - loss: 703.3804 - val_loss: 778.3696
Epoch 50/100
 - 0s - loss: 680.1841 - val_loss: 755.9723
Epoch 51/100
 - 0s - loss: 657.6365 - val_loss: 733.6218
Epoch 52/100
 - 0s - loss: 635.5211 - val_loss: 711.6787
Epoch 53/100
 - 0s - loss: 613.6259 - val_loss: 690.3949
Epoch 54/100
 - 0s - loss: 592.8975 - val_loss: 668.7206
Epoch 55/100
 - 0s - loss: 571.5696 - val_loss: 648.3338
Epoch 56/100
 - 0s - loss: 551.5720 - val_loss: 628.0620
Epoch 57/100
 - 0s - loss: 531.8392 - val_loss: 608.2903
Epoch 58/100
 - 0s - loss: 513.1149 - val_loss: 588.2361
Epoch 59/100
 - 0s - loss: 494.

Epoch 84/100
 - 0s - loss: 259.7238 - val_loss: 230.6637
Epoch 85/100
 - 0s - loss: 255.2517 - val_loss: 226.5933
Epoch 86/100
 - 0s - loss: 250.8149 - val_loss: 222.8801
Epoch 87/100
 - 0s - loss: 246.7545 - val_loss: 219.3604
Epoch 88/100
 - 0s - loss: 242.8700 - val_loss: 216.1238
Epoch 89/100
 - 0s - loss: 239.2357 - val_loss: 213.0259
Epoch 90/100
 - 0s - loss: 235.8248 - val_loss: 210.1087
Epoch 91/100
 - 0s - loss: 232.5667 - val_loss: 207.2184
Epoch 92/100
 - 0s - loss: 229.4502 - val_loss: 204.7007
Epoch 93/100
 - 0s - loss: 226.4836 - val_loss: 202.3987
Epoch 94/100
 - 0s - loss: 223.7885 - val_loss: 200.0209
Epoch 95/100
 - 0s - loss: 221.0241 - val_loss: 197.7060
Epoch 96/100
 - 0s - loss: 218.6057 - val_loss: 195.5359
Epoch 97/100
 - 0s - loss: 216.1072 - val_loss: 193.4070
Epoch 98/100
 - 0s - loss: 213.7497 - val_loss: 191.2816
Epoch 99/100
 - 0s - loss: 211.4827 - val_loss: 189.5838
Epoch 100/100
 - 0s - loss: 209.3275 - val_loss: 187.7838
2
Train on 504 samples, valida

Epoch 25/100
 - 0s - loss: 1235.2052 - val_loss: 1136.2673
Epoch 26/100
 - 0s - loss: 1214.5540 - val_loss: 1116.2064
Epoch 27/100
 - 0s - loss: 1193.0009 - val_loss: 1096.2029
Epoch 28/100
 - 0s - loss: 1171.7841 - val_loss: 1075.3516
Epoch 29/100
 - 0s - loss: 1149.9092 - val_loss: 1054.0305
Epoch 30/100
 - 0s - loss: 1127.4144 - val_loss: 1032.9289
Epoch 31/100
 - 0s - loss: 1104.8318 - val_loss: 1011.6857
Epoch 32/100
 - 0s - loss: 1081.8731 - val_loss: 990.3794
Epoch 33/100
 - 0s - loss: 1059.3612 - val_loss: 968.1257
Epoch 34/100
 - 0s - loss: 1035.6457 - val_loss: 946.3016
Epoch 35/100
 - 0s - loss: 1012.6093 - val_loss: 923.8702
Epoch 36/100
 - 0s - loss: 988.9212 - val_loss: 901.8013
Epoch 37/100
 - 0s - loss: 965.5004 - val_loss: 879.0542
Epoch 38/100
 - 0s - loss: 941.7578 - val_loss: 856.3655
Epoch 39/100
 - 0s - loss: 917.7923 - val_loss: 833.9304
Epoch 40/100
 - 0s - loss: 893.9711 - val_loss: 811.3418
Epoch 41/100
 - 0s - loss: 870.2341 - val_loss: 788.3746
Epoch 42/100


Epoch 67/100
 - 0s - loss: 399.3730 - val_loss: 337.7700
Epoch 68/100
 - 0s - loss: 389.2828 - val_loss: 329.0396
Epoch 69/100
 - 0s - loss: 379.7707 - val_loss: 320.7470
Epoch 70/100
 - 0s - loss: 370.5551 - val_loss: 312.7197
Epoch 71/100
 - 0s - loss: 361.9115 - val_loss: 305.0503
Epoch 72/100
 - 0s - loss: 353.3822 - val_loss: 297.9775
Epoch 73/100
 - 0s - loss: 345.7693 - val_loss: 290.8478
Epoch 74/100
 - 0s - loss: 337.8427 - val_loss: 284.3850
Epoch 75/100
 - 0s - loss: 330.6180 - val_loss: 278.0288
Epoch 76/100
 - 0s - loss: 323.6898 - val_loss: 271.9554
Epoch 77/100
 - 0s - loss: 316.8718 - val_loss: 266.4337
Epoch 78/100
 - 0s - loss: 310.5696 - val_loss: 261.0875
Epoch 79/100
 - 0s - loss: 304.5859 - val_loss: 255.7294
Epoch 80/100
 - 0s - loss: 298.5655 - val_loss: 250.9397
Epoch 81/100
 - 0s - loss: 293.0803 - val_loss: 246.2990
Epoch 82/100
 - 0s - loss: 287.7497 - val_loss: 242.0264
Epoch 83/100
 - 0s - loss: 282.6173 - val_loss: 237.8142
Epoch 84/100
 - 0s - loss: 277.

...

Epoch 26/100
 - 0s - loss: 1106.2276 - val_loss: 1076.5538
Epoch 27/100
 - 0s - loss: 1080.4117 - val_loss: 1052.8572
Epoch 28/100
 - 0s - loss: 1054.6336 - val_loss: 1028.2245
Epoch 29/100
 - 0s - loss: 1028.1823 - val_loss: 1003.5639
Epoch 30/100
 - 0s - loss: 1001.5829 - val_loss: 978.0851
Epoch 31/100
 - 0s - loss: 974.8117 - val_loss: 952.6649
Epoch 32/100
 - 0s - loss: 948.2931 - val_loss: 927.0421
Epoch 33/100
 - 0s - loss: 921.2545 - val_loss: 901.6823
Epoch 34/100
 - 0s - loss: 894.5654 - val_loss: 876.2903
Epoch 35/100
 - 0s - loss: 868.1689 - val_loss: 850.5378
Epoch 36/100
 - 0s - loss: 841.7812 - val_loss: 825.7477
Epoch 37/100
 - 0s - loss: 815.8211 - val_loss: 800.4103
Epoch 38/100
 - 0s - loss: 790.2457 - val_loss: 775.2029
Epoch 39/100
 - 0s - loss: 764.7819 - val_loss: 750.5261
Epoch 40/100
 - 0s - loss: 739.7489 - val_loss: 726.3195
Epoch 41/100
 - 0s - loss: 715.6871 - val_loss: 702.1699
Epoch 42/100
 - 0s - loss: 691.4616 - val_loss: 678.7079
Epoch 43/100
 - 0s - l

Epoch 68/100
 - 0s - loss: 419.3025 - val_loss: 393.5832
Epoch 69/100
 - 0s - loss: 407.0288 - val_loss: 382.0635
Epoch 70/100
 - 0s - loss: 395.6273 - val_loss: 370.7340
Epoch 71/100
 - 0s - loss: 384.0361 - val_loss: 360.3315
Epoch 72/100
 - 0s - loss: 373.8990 - val_loss: 349.4069
Epoch 73/100
 - 0s - loss: 363.0613 - val_loss: 339.6333
Epoch 74/100
 - 0s - loss: 353.1720 - val_loss: 330.1741
Epoch 75/100
 - 0s - loss: 343.7659 - val_loss: 321.2178
Epoch 76/100
 - 0s - loss: 334.8177 - val_loss: 312.5532
Epoch 77/100
 - 0s - loss: 326.4709 - val_loss: 304.1418
Epoch 78/100
 - 0s - loss: 318.0391 - val_loss: 296.3182
Epoch 79/100
 - 0s - loss: 310.0883 - val_loss: 289.3524
Epoch 80/100
 - 0s - loss: 302.9933 - val_loss: 282.0613
Epoch 81/100
 - 0s - loss: 295.7394 - val_loss: 275.5425
Epoch 82/100
 - 0s - loss: 289.1689 - val_loss: 269.2324
Epoch 83/100
 - 0s - loss: 282.9754 - val_loss: 262.9899
Epoch 84/100
 - 0s - loss: 276.8912 - val_loss: 257.3241
Epoch 85/100
 - 0s - loss: 270.

Epoch 9/100
 - 0s - loss: 1520.8217 - val_loss: 1308.0533
Epoch 10/100
 - 0s - loss: 1509.5234 - val_loss: 1297.6890
Epoch 11/100
 - 0s - loss: 1497.6462 - val_loss: 1287.0856
Epoch 12/100
 - 0s - loss: 1485.3787 - val_loss: 1276.0013
Epoch 13/100
 - 0s - loss: 1472.6357 - val_loss: 1264.1897
Epoch 14/100
 - 0s - loss: 1459.1214 - val_loss: 1251.9424
Epoch 15/100
 - 0s - loss: 1445.2013 - val_loss: 1238.8231
Epoch 16/100
 - 0s - loss: 1430.1925 - val_loss: 1225.2247
Epoch 17/100
 - 0s - loss: 1414.5631 - val_loss: 1210.8397
Epoch 18/100
 - 0s - loss: 1397.9658 - val_loss: 1195.7481
Epoch 19/100
 - 0s - loss: 1380.4361 - val_loss: 1180.1542
Epoch 20/100
 - 0s - loss: 1362.5640 - val_loss: 1163.3572
Epoch 21/100
 - 0s - loss: 1343.2545 - val_loss: 1146.0601
Epoch 22/100
 - 0s - loss: 1323.3704 - val_loss: 1128.1766
Epoch 23/100
 - 0s - loss: 1302.8897 - val_loss: 1109.3866
Epoch 24/100
 - 0s - loss: 1281.2833 - val_loss: 1090.3812
Epoch 25/100
 - 0s - loss: 1259.1480 - val_loss: 1070.928

 - 0s - loss: 575.1674 - val_loss: 571.0887
Epoch 51/100
 - 0s - loss: 556.7091 - val_loss: 552.7917
Epoch 52/100
 - 0s - loss: 538.7796 - val_loss: 535.0298
Epoch 53/100
 - 0s - loss: 521.5986 - val_loss: 517.3481
Epoch 54/100
 - 0s - loss: 504.4190 - val_loss: 500.9281
Epoch 55/100
 - 0s - loss: 488.2224 - val_loss: 484.7895
Epoch 56/100
 - 0s - loss: 472.6595 - val_loss: 468.9923
Epoch 57/100
 - 0s - loss: 457.2688 - val_loss: 454.1066
Epoch 58/100
 - 0s - loss: 442.9126 - val_loss: 439.3923
Epoch 59/100
 - 0s - loss: 428.6765 - val_loss: 425.6013
Epoch 60/100
 - 0s - loss: 415.3352 - val_loss: 412.1181
Epoch 61/100
 - 0s - loss: 402.5572 - val_loss: 398.7908
Epoch 62/100
 - 0s - loss: 389.9575 - val_loss: 386.4916
Epoch 63/100
 - 0s - loss: 378.0008 - val_loss: 374.6250
Epoch 64/100
 - 0s - loss: 366.4918 - val_loss: 363.5185
Epoch 65/100
 - 0s - loss: 356.0403 - val_loss: 352.3257
Epoch 66/100
 - 0s - loss: 345.3977 - val_loss: 342.2205
Epoch 67/100
 - 0s - loss: 335.7660 - val_lo

[71622746.95365463,
 3292859.043474331,
 26656288.502041228,
 53445241.73517395,
 53860043.82874816,
 51750847.14016352,
 33885294.1122228,
 51784685.46778528,
 26368551.749751203,
 9500816.15086799,
 13595145.66756739,
 29689859.595265016,
 10766308.97758007,
 85424306.81225231,
 24503571.89495218,
 20197163.474327434,
 48745097.1370856,
 50348823.178170085,
 9943504.411037581,
 35740569.99725406,
 50772364.36535018,
 5098537.129931205,
 9838702.731685672,
 19547748.79737179,
 22354761.80474104,
 1103865.4728200089,
 3771500.828853517,
 17161408.158776075,
 15935780.002758102,
 31143702.486977696,
 52384667.790193014,
 1838822.393415808,
 16709869.520353206,
 21070998.81337097,
 38552553.92552732,
 32455875.53794904,
 6158948.886217906,
 5960605.281587849,
 67413584.2382201,
 20005018.157660242,
 10399231.738182994,
 11111375.052589266,
 29454664.83609753,
 111921652.15525986,
 19983297.144280024,
 9590501.145727899,
 11369849.085843585,
 9681058.071651727,
 20306265.30253091,
 199719

In [36]:
mean_squared_error_array3

[71622746.95365463,
 3292859.043474331,
 26656288.502041228,
 53445241.73517395,
 53860043.82874816,
 51750847.14016352,
 33885294.1122228,
 51784685.46778528,
 26368551.749751203,
 9500816.15086799,
 13595145.66756739,
 29689859.595265016,
 10766308.97758007,
 85424306.81225231,
 24503571.89495218,
 20197163.474327434,
 48745097.1370856,
 50348823.178170085,
 9943504.411037581,
 35740569.99725406,
 50772364.36535018,
 5098537.129931205,
 9838702.731685672,
 19547748.79737179,
 22354761.80474104,
 1103865.4728200089,
 3771500.828853517,
 17161408.158776075,
 15935780.002758102,
 31143702.486977696,
 52384667.790193014,
 1838822.393415808,
 16709869.520353206,
 21070998.81337097,
 38552553.92552732,
 32455875.53794904,
 6158948.886217906,
 5960605.281587849,
 67413584.2382201,
 20005018.157660242,
 10399231.738182994,
 11111375.052589266,
 29454664.83609753,
 111921652.15525986,
 19983297.144280024,
 9590501.145727899,
 11369849.085843585,
 9681058.071651727,
 20306265.30253091,
 199719

In [37]:
print("mean:",mean(mean_squared_error_array3))
print("sd:",pstdev(mean_squared_error_array3))

mean: 28083817.287615553
sd: 23038430.0127295


Model built on normalised data with increased number of hideen layers

In [38]:
# define regression model
def regression_model2():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    #model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

mean_squared_error_array4=[]
for i in range(50):
    #predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    predictors_norm_train, predictors_norm_test, target_train, target_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model = regression_model2()
    model.fit(predictors_norm_train, target_train, validation_split=0.3, epochs=50, verbose=2)
    target_predicted=model.predict(predictors_test)
    target_predicted
    results = model.evaluate(predictors_norm_test,target_test,verbose=0)
    #print('test loss, test acc:', results)
    mean_squared_error_array4.append(mean_squared_error(target_test, target_predicted))
    print(i)
mean_squared_error_array4
mean(mean_squared_error_array4)

Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 1554.1973 - val_loss: 1632.2094
Epoch 2/50
 - 0s - loss: 1545.9305 - val_loss: 1624.5486
Epoch 3/50
 - 0s - loss: 1538.6030 - val_loss: 1616.5972
Epoch 4/50
 - 0s - loss: 1529.8542 - val_loss: 1606.8282
Epoch 5/50
 - 0s - loss: 1518.6936 - val_loss: 1593.4941
Epoch 6/50
 - 0s - loss: 1503.3735 - val_loss: 1575.2177
Epoch 7/50
 - 0s - loss: 1482.4465 - val_loss: 1549.2865
Epoch 8/50
 - 0s - loss: 1452.9639 - val_loss: 1513.0700
Epoch 9/50
 - 0s - loss: 1411.5716 - val_loss: 1462.8734
Epoch 10/50
 - 0s - loss: 1355.1257 - val_loss: 1392.4672
Epoch 11/50
 - 0s - loss: 1276.4630 - val_loss: 1292.5865
Epoch 12/50
 - 0s - loss: 1167.3568 - val_loss: 1161.1109
Epoch 13/50
 - 0s - loss: 1029.3242 - val_loss: 994.5037
Epoch 14/50
 - 0s - loss: 860.6398 - val_loss: 802.8023
Epoch 15/50
 - 0s - loss: 675.6523 - val_loss: 606.3505
Epoch 16/50
 - 0s - loss: 500.3792 - val_loss: 433.4652
Epoch 17/50
 - 0s - loss: 359.6095 - val_l

Epoch 44/50
 - 0s - loss: 159.8606 - val_loss: 140.6812
Epoch 45/50
 - 0s - loss: 157.8656 - val_loss: 139.1940
Epoch 46/50
 - 0s - loss: 156.0914 - val_loss: 138.8776
Epoch 47/50
 - 0s - loss: 154.4837 - val_loss: 137.3229
Epoch 48/50
 - 0s - loss: 152.6358 - val_loss: 136.1611
Epoch 49/50
 - 0s - loss: 151.1913 - val_loss: 135.2474
Epoch 50/50
 - 0s - loss: 149.5602 - val_loss: 134.6947
2
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 1487.0301 - val_loss: 1658.8266
Epoch 2/50
 - 0s - loss: 1472.2265 - val_loss: 1641.9985
Epoch 3/50
 - 0s - loss: 1454.3474 - val_loss: 1620.3445
Epoch 4/50
 - 0s - loss: 1431.5119 - val_loss: 1591.0658
Epoch 5/50
 - 0s - loss: 1400.2629 - val_loss: 1553.0621
Epoch 6/50
 - 0s - loss: 1359.3399 - val_loss: 1502.5231
Epoch 7/50
 - 0s - loss: 1306.7865 - val_loss: 1434.1238
Epoch 8/50
 - 0s - loss: 1234.8091 - val_loss: 1344.7256
Epoch 9/50
 - 0s - loss: 1142.0286 - val_loss: 1224.5955
Epoch 10/50
 - 0s - loss: 1018.7597 - val_loss:

Epoch 38/50
 - 0s - loss: 154.8190 - val_loss: 155.8199
Epoch 39/50
 - 0s - loss: 153.5556 - val_loss: 154.5871
Epoch 40/50
 - 0s - loss: 152.3700 - val_loss: 153.6922
Epoch 41/50
 - 0s - loss: 150.8003 - val_loss: 153.3098
Epoch 42/50
 - 0s - loss: 149.7060 - val_loss: 151.6503
Epoch 43/50
 - 0s - loss: 148.6474 - val_loss: 151.6370
Epoch 44/50
 - 0s - loss: 147.4063 - val_loss: 149.5088
Epoch 45/50
 - 0s - loss: 145.9920 - val_loss: 148.9264
Epoch 46/50
 - 0s - loss: 144.9958 - val_loss: 148.3122
Epoch 47/50
 - 0s - loss: 143.8940 - val_loss: 148.2064
Epoch 48/50
 - 0s - loss: 142.8460 - val_loss: 147.2556
Epoch 49/50
 - 0s - loss: 141.9402 - val_loss: 145.5491
Epoch 50/50
 - 0s - loss: 140.8094 - val_loss: 145.4558
5
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 1503.0652 - val_loss: 1503.2409
Epoch 2/50
 - 0s - loss: 1491.9177 - val_loss: 1494.0650
Epoch 3/50
 - 0s - loss: 1482.2047 - val_loss: 1484.8815
Epoch 4/50
 - 0s - loss: 1471.6585 - val_loss: 1472.9

Epoch 32/50
 - 0s - loss: 160.3486 - val_loss: 172.1179
Epoch 33/50
 - 0s - loss: 158.6185 - val_loss: 170.0418
Epoch 34/50
 - 0s - loss: 156.8814 - val_loss: 168.2497
Epoch 35/50
 - 0s - loss: 155.0671 - val_loss: 166.7007
Epoch 36/50
 - 0s - loss: 153.6296 - val_loss: 165.2223
Epoch 37/50
 - 0s - loss: 152.1145 - val_loss: 163.1732
Epoch 38/50
 - 0s - loss: 150.7528 - val_loss: 161.3152
Epoch 39/50
 - 0s - loss: 149.2899 - val_loss: 160.2921
Epoch 40/50
 - 0s - loss: 147.8890 - val_loss: 159.1656
Epoch 41/50
 - 0s - loss: 146.7523 - val_loss: 157.6656
Epoch 42/50
 - 0s - loss: 145.5979 - val_loss: 156.1795
Epoch 43/50
 - 0s - loss: 144.6074 - val_loss: 155.3998
Epoch 44/50
 - 0s - loss: 143.2329 - val_loss: 153.9492
Epoch 45/50
 - 0s - loss: 142.6202 - val_loss: 152.9391
Epoch 46/50
 - 0s - loss: 141.5105 - val_loss: 152.1519
Epoch 47/50
 - 0s - loss: 140.3612 - val_loss: 150.6583
Epoch 48/50
 - 0s - loss: 139.1989 - val_loss: 149.4869
Epoch 49/50
 - 0s - loss: 138.8087 - val_loss: 1

 - 0s - loss: 169.7849 - val_loss: 223.4927
Epoch 26/50
 - 0s - loss: 166.6821 - val_loss: 221.1648
Epoch 27/50
 - 0s - loss: 164.1473 - val_loss: 218.9496
Epoch 28/50
 - 0s - loss: 161.5591 - val_loss: 216.0635
Epoch 29/50
 - 0s - loss: 159.4394 - val_loss: 213.5156
Epoch 30/50
 - 0s - loss: 157.5531 - val_loss: 211.3465
Epoch 31/50
 - 0s - loss: 155.8734 - val_loss: 209.0425
Epoch 32/50
 - 0s - loss: 153.6479 - val_loss: 208.0470
Epoch 33/50
 - 0s - loss: 152.1569 - val_loss: 205.3430
Epoch 34/50
 - 0s - loss: 150.3766 - val_loss: 203.9801
Epoch 35/50
 - 0s - loss: 149.2467 - val_loss: 201.8987
Epoch 36/50
 - 0s - loss: 147.8571 - val_loss: 201.0966
Epoch 37/50
 - 0s - loss: 146.4983 - val_loss: 198.3136
Epoch 38/50
 - 0s - loss: 145.1196 - val_loss: 197.0497
Epoch 39/50
 - 0s - loss: 143.9656 - val_loss: 195.7931
Epoch 40/50
 - 0s - loss: 142.9046 - val_loss: 194.1878
Epoch 41/50
 - 0s - loss: 141.6085 - val_loss: 193.0520
Epoch 42/50
 - 0s - loss: 140.6159 - val_loss: 191.5640
Epoc

...

Epoch 13/50
 - 0s - loss: 578.5845 - val_loss: 488.6083
Epoch 14/50
 - 0s - loss: 473.1320 - val_loss: 399.4795
Epoch 15/50
 - 0s - loss: 390.7829 - val_loss: 334.2812
Epoch 16/50
 - 0s - loss: 333.6130 - val_loss: 292.3065
Epoch 17/50
 - 0s - loss: 294.1282 - val_loss: 270.0221
Epoch 18/50
 - 0s - loss: 271.4459 - val_loss: 256.1394
Epoch 19/50
 - 0s - loss: 255.1499 - val_loss: 246.8025
Epoch 20/50
 - 0s - loss: 242.6118 - val_loss: 239.8821
Epoch 21/50
 - 0s - loss: 232.4415 - val_loss: 233.3661
Epoch 22/50
 - 0s - loss: 223.6801 - val_loss: 227.8667
Epoch 23/50
 - 0s - loss: 215.6709 - val_loss: 222.9205
Epoch 24/50
 - 0s - loss: 208.6286 - val_loss: 218.5752
Epoch 25/50
 - 0s - loss: 202.3271 - val_loss: 214.1202
Epoch 26/50
 - 0s - loss: 196.5061 - val_loss: 210.2927
Epoch 27/50
 - 0s - loss: 191.3228 - val_loss: 206.8931
Epoch 28/50
 - 0s - loss: 186.4000 - val_loss: 203.7462
Epoch 29/50
 - 0s - loss: 181.4630 - val_loss: 199.3311
Epoch 30/50
 - 0s - loss: 177.1580 - val_loss: 1

Epoch 7/50
 - 0s - loss: 1484.2709 - val_loss: 1427.7419
Epoch 8/50
 - 0s - loss: 1450.4834 - val_loss: 1389.9580
Epoch 9/50
 - 0s - loss: 1404.5132 - val_loss: 1339.0645
Epoch 10/50
 - 0s - loss: 1343.5117 - val_loss: 1273.0856
Epoch 11/50
 - 0s - loss: 1263.9341 - val_loss: 1189.4585
Epoch 12/50
 - 0s - loss: 1163.0251 - val_loss: 1083.4324
Epoch 13/50
 - 0s - loss: 1036.0440 - val_loss: 953.3488
Epoch 14/50
 - 0s - loss: 889.5959 - val_loss: 806.0406
Epoch 15/50
 - 0s - loss: 730.5219 - val_loss: 659.3126
Epoch 16/50
 - 0s - loss: 582.3544 - val_loss: 523.0531
Epoch 17/50
 - 0s - loss: 457.8698 - val_loss: 413.2913
Epoch 18/50
 - 0s - loss: 364.6440 - val_loss: 335.3453
Epoch 19/50
 - 0s - loss: 302.5874 - val_loss: 286.3172
Epoch 20/50
 - 0s - loss: 266.0206 - val_loss: 254.8567
Epoch 21/50
 - 0s - loss: 240.5934 - val_loss: 237.3097
Epoch 22/50
 - 0s - loss: 225.9643 - val_loss: 225.7730
Epoch 23/50
 - 0s - loss: 215.5198 - val_loss: 218.1315
Epoch 24/50
 - 0s - loss: 208.6548 - v

Epoch 2/50
 - 0s - loss: 1629.0729 - val_loss: 1512.8828
Epoch 3/50
 - 0s - loss: 1606.4535 - val_loss: 1491.5145
Epoch 4/50
 - 0s - loss: 1585.2674 - val_loss: 1470.0457
Epoch 5/50
 - 0s - loss: 1561.8972 - val_loss: 1445.3426
Epoch 6/50
 - 0s - loss: 1533.6811 - val_loss: 1412.5255
Epoch 7/50
 - 0s - loss: 1495.9872 - val_loss: 1369.2589
Epoch 8/50
 - 0s - loss: 1446.2775 - val_loss: 1312.9404
Epoch 9/50
 - 0s - loss: 1384.0963 - val_loss: 1242.4079
Epoch 10/50
 - 0s - loss: 1301.8493 - val_loss: 1157.3682
Epoch 11/50
 - 0s - loss: 1203.1642 - val_loss: 1048.6942
Epoch 12/50
 - 0s - loss: 1080.1360 - val_loss: 918.3525
Epoch 13/50
 - 0s - loss: 934.1555 - val_loss: 769.3130
Epoch 14/50
 - 0s - loss: 773.2777 - val_loss: 610.6245
Epoch 15/50
 - 0s - loss: 609.4608 - val_loss: 460.0006
Epoch 16/50
 - 0s - loss: 461.9938 - val_loss: 339.4027
Epoch 17/50
 - 0s - loss: 353.7271 - val_loss: 260.5073
Epoch 18/50
 - 0s - loss: 286.1614 - val_loss: 220.8243
Epoch 19/50
 - 0s - loss: 250.0159 

 - 0s - loss: 142.2661 - val_loss: 177.8231
Epoch 47/50
 - 0s - loss: 141.2152 - val_loss: 175.1535
Epoch 48/50
 - 0s - loss: 139.9671 - val_loss: 174.9423
Epoch 49/50
 - 0s - loss: 139.2608 - val_loss: 175.0707
Epoch 50/50
 - 0s - loss: 138.2572 - val_loss: 173.8359
48
Train on 504 samples, validate on 217 samples
Epoch 1/50
 - 0s - loss: 1548.9409 - val_loss: 1555.6004
Epoch 2/50
 - 0s - loss: 1534.1577 - val_loss: 1539.1301
Epoch 3/50
 - 0s - loss: 1515.8400 - val_loss: 1519.2147
Epoch 4/50
 - 0s - loss: 1494.1926 - val_loss: 1494.7333
Epoch 5/50
 - 0s - loss: 1466.6508 - val_loss: 1465.1701
Epoch 6/50
 - 0s - loss: 1433.2447 - val_loss: 1427.4464
Epoch 7/50
 - 0s - loss: 1390.9509 - val_loss: 1379.8437
Epoch 8/50
 - 0s - loss: 1337.4831 - val_loss: 1319.3731
Epoch 9/50
 - 0s - loss: 1268.2989 - val_loss: 1243.3331
Epoch 10/50
 - 0s - loss: 1182.5685 - val_loss: 1146.0409
Epoch 11/50
 - 0s - loss: 1074.7968 - val_loss: 1026.0636
Epoch 12/50
 - 0s - loss: 946.3612 - val_loss: 891.384

84412242.26710251

In [39]:
print("mean:",mean(mean_squared_error_array4))
print("sd:",pstdev(mean_squared_error_array4))

mean: 84412242.26710251
sd: 49266355.229643844


In [40]:
print("mean1:",mean(mean_squared_error_array))
print("sd1:",pstdev(mean_squared_error_array))
print("mean2:",mean(mean_squared_error_array2))
print("sd2:",pstdev(mean_squared_error_array2))
print("mean3:",mean(mean_squared_error_array3))
print("sd3:",pstdev(mean_squared_error_array3))
print("mean4:",mean(mean_squared_error_array4))
print("sd4:",pstdev(mean_squared_error_array4))


mean1: 561.1341654461006
sd1: 544.5242683332228
mean2: 7125312.897080838
sd2: 5740924.124464269
mean3: 28083817.287615553
sd3: 23038430.0127295
mean4: 84412242.26710251
sd4: 49266355.229643844


In [41]:
print("Case A: mean and the standard deviation of the mean squared errors is", mean(mean_squared_error_array))
print("and",pstdev(mean_squared_error_array))

Case A: mean and the standard deviation of the mean squared errors is 561.1341654461006
and 544.5242683332228


In [42]:
print("Case B: mean  of the mean squared errors increased from", mean(mean_squared_error_array))
print("to",mean(mean_squared_error_array2))

Case B: mean  of the mean squared errors increased from 561.1341654461006
to 7125312.897080838


In [43]:
print("Case C: mean  of the mean squared errors increased from", mean(mean_squared_error_array2))
print("to",mean(mean_squared_error_array3))

Case C: mean  of the mean squared errors increased from 7125312.897080838
to 28083817.287615553


In [44]:
print("Case D: mean  of the mean squared errors increased from", mean(mean_squared_error_array2))
print("to",mean(mean_squared_error_array4))

Case D: mean  of the mean squared errors increased from 7125312.897080838
to 84412242.26710251
